In [1]:
from keras.models import load_model
model=load_model('model3.h5')
from tkinter import filedialog
import matplotlib.pyplot as plt
import numpy as np
import copy
import cv2
import os
import pygame
gesture_actions = {
    'NONE': 'No Action',
    'ONE': 'Next Song',
    'TWO': 'Previous Song',
    'THREE': 'Volume Up',
    'FOUR': 'Volume Down',
    'FIVE': 'Pause or Unpause'
    # Add more mappings if you have more classes
}
dir1=os.getcwd()
songslist=[]
n1=0
count=0
bckgrd = None
n=0
y,x1,x2,x3,x4,x5,x6=0,0,0,0,0,0,0
shmsk=0
x0, y0, width = 335, 105, 300
dataColor = (0,255,0)
flag=0
classes = 'NONE ONE TWO THREE FOUR FIVE'.split()
print("classes done")
def upcomingsong():
    global n
    n+=1
    if(count==n):
        n=0
    pygame.mixer.music.load(songslist[n])
    pygame.mixer.music.play()
    print("next song gesture received")
print("classes done")
def pausesong():
    global flag
    if flag==0:
        pygame.mixer.music.pause()
        print("music pause") 
        flag=1
    else:
        flag=0
        pygame.mixer.music.unpause()
        print("unpause")
print("some functions done")
def preceedingsong():
    global n
    n-=1
    if(n==-1):
        n=count-1
    pygame.mixer.music.load(songslist[n])
    pygame.mixer.music.play()
    print("prev song gesture received")
def volup():
    pygame.mixer.music.set_volume(2)
    print("volume up gesture received")
def voldown():
    pygame.mixer.music.set_volume(0.5)
    print("volume down gesture received")
def load():
    global count
    directory =os.getcwd()
    os.chdir(directory)
    for files in os.listdir(directory):
        if files.endswith(".mp3"):
            songslist.append(files)
            count=count+1
    pygame.mixer.init()
    pygame.mixer.music.load(songslist[0])
    pygame.mixer.music.play()
print("all functions done")
load()
print("load songs done")
def runningavg(image, acc=0.5):
    global bckgrd
    if bckgrd is None:
        bckgrd = image.copy().astype("float")
        return
    cv2.accumulateWeighted(image, bckgrd, 0.5)
def mask(img,img1):
    global bckgrd
    diff = cv2.absdiff(bckgrd.astype("uint8"), img1)
    cv2.imshow("diff", diff)
    thresholded = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)[1]
    cv2.imshow("thresholded", thresholded)
    cnts= cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(cnts) == 0:
        return
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.GaussianBlur(img, (7,7), 3)
        img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
        #cv2.imshow("img",img)
        r, thr = cv2.threshold(img, 25, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        cv2.imshow("thr", thr)
        return thr
cam = cv2.VideoCapture(0)
cv2.namedWindow('Original', cv2.WINDOW_NORMAL)
numofframes=0
while True:
    # Get camera frame
    r, image = cam.read()
    image = cv2.flip(image, 1) # mirror
    image1 = copy.deepcopy(image)
    cv2.rectangle(image, (x0,y0), (x0+width-1,y0+width-1), dataColor, 12)
    roi = image[y0:y0+width,x0:x0+width]
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    cv2.imshow('gray',gray)
    if numofframes <= 30:
            runningavg(gray, acc=0.5)
            if numofframes == 30:
                print ("running average done.")
    else:
        roi = mask(roi,gray)
        if roi is not None:
            img = np.float32(roi)/255.
            img = np.expand_dims(img, axis=0)
            img = np.expand_dims(img, axis=-1)
            pred = classes[np.argmax(model.predict(img)[0])]
            if pred in gesture_actions:
                action_text = gesture_actions[pred]
                display_text = f"Gesture: {pred}, Action: {action_text}"
            else:
                display_text = f"Gesture: {pred}, Action: Unknown"
            cv2.putText(image, display_text, (10,50+2*45),cv2.FONT_HERSHEY_SIMPLEX, 1.0, (245,210,65), 2, 1)
            
            n1=n1+1
            if(pred=='ONE'):
                x2+=1
            if(pred=='TWO'):
                x3+=1
            if(pred=='THREE'):
                x4+=1
            if(pred=='FOUR'):
                x5+=1
            if(pred=='FIVE'):
                x6+=1
            if(n1==100):
                print(x1,x2,x3,x4,x5,x6,sep=" ")
                y=max(x1,x2,x3,x4,x5,x6)
                #print('y=')
                #print(y)
                n1=0
                if(y==x2 and y!=0):
                    upcomingsong()
                    x1,x2,x3,x4,x5,x6=0,0,0,0,0,0
                if(y==x3 and y!=0):
                    preceedingsong()
                    x1,x2,x3,x4,x5,x6=0,0,0,0,0,0
                if(y==x4 and y!=0):
                    volup()
                    x1,x2,x3,x4,x5,x6=0,0,0,0,0,0
                if(y==x5 and y!=0):
                    voldown()
                    x1,x2,x3,x4,x5,x6=0,0,0,0,0,0
                if(y==x6 and y!=0):
                    pausesong()
                    x1,x2,x3,x4,x5,x6=0,0,0,0,0,0
            key = cv2.waitKey(10) & 0xff
            #cv2.putText(image, pred, (10,50+2*45),cv2.FONT_HERSHEY_SIMPLEX, 1.0, (245,210,65), 2, 1)
            cv2.imshow('Original', image)
            # use q key to close the program
            if key == ord('q'):
                pygame.mixer.music.stop() 
                break
    numofframes+=1
cam.release()
os.chdir(dir1)
cv2.destroyAllWindows()




pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
classes done
classes done
some functions done
all functions done
load songs done
running average done.
1/1 [==============================] - 0s 48ms/step
0 58 0 0 0 0
next song gesture received
1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 43ms/step
0 22 0 0 0 0
next song gesture received
1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 45ms/step
0 6 32 1 0 0
prev song gesture received
1/1 [==============================] - 0s 41ms/step
0 9 56 0 0 0
prev song gesture received
1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 44ms/step
0 51 39 2 4 0
next song gesture received
1/1 [==============================] - 0s 49ms/step
0 70 0 0 0 0
next song gesture received
1/1 [==============================] - 0s 45ms/step


1/1 [==============================] - 0s 46ms/step
0 30 70 0 0 0
prev song gesture received
1/1 [==============================] - 0s 44ms/step


...

1/1 [==============================] - 0s 45ms/step
0 15 0 0 0 0
next song gesture received
1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 42ms/step
0 11 4 0 0 0
next song gesture received
1/1 [==============================] - 0s 43ms/step
0 24 64 0 0 0
prev song gesture received
1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 48ms/step
0 1 9 38 0 0
volume up gesture received
1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 45ms/step
0 0 1 41 58 0
volume down gesture received
1/1 [==============================] - 0s 41ms/step
0 0 0 1 83 16
volume down gesture received
1/1 [==============================] - 0s 46ms/step


1/1 [==============================] - 0s 43ms/step
0 0 7 0 73 17
volume down gesture received
1/1 [==============================] - 0s 135ms/step
